In [ ]:
import numpy as np
import pandas as pd

In [ ]:
data = pd.read_csv('train.tsv',sep='\t',header=None)
data.columns = ['words','pos']

In [ ]:
data=data.astype(str)

In [ ]:
lengths = data["words"].str.len()
argmax = np.where(lengths == lengths.max())[0]
data.iloc[argmax]

,words,pos
2286166,\tN\nní\tN\ndóigh\tN\nliom\tN\ngo\tN\nfuil\tU\...,N


In [ ]:
len(data['words'][2286166])

57291

In [ ]:
def bigram(df):
  gram = []
  for i in range(0,len(df['words'])-1):
    gram.append(df['words'][i]+' '+df['words'][i+1])
  return gram


In [ ]:
k = bigram(data)

In [ ]:
l = data['pos'][1:].tolist()

In [ ]:
# BIGRAM TUPLE LIST



lis=[]
for i in range(len(k)):
   lis.append((k[i],l[i]))
  


In [ ]:
# UNIGRAM TUPLE LIST

m = data['words'].tolist()
n = data['pos'].tolist()
liss=[]
for i in range(len(m)):
   liss.append((m[i],n[i]))
  


In [ ]:
test_set = liss[4551353:]
train_set = liss[:4551353] 

In [ ]:
  # j = df['pos'][1:].tolist()
  # b_gram = [ x + y for x, y in zip(gram, j)]

In [ ]:
# data = data.groupby((np.repeat(np.arange(len(data)), 2)[:len(data)])).agg(' '.join)
# data['pos'] = data['pos'].str[2:]
# data['pos'][2528529] = 'N'

In [ ]:
(data['pos']=='N').sum()

4339849

In [ ]:
#use set datatype to check how many unique tags are present in training data
tags = {tag for word,tag in train_set}
print(len(tags))
print(tags)
 
# check total words in vocabulary
vocab = {word for word,tag in train_set}
print(len(vocab))

5
{'U', 'T', 'N', 'S', 'H'}
118706


In [ ]:
# compute Emission Probability
def word_given_tag(word, tag, train_bag = train_set):
    tag_list = [pair for pair in train_bag if pair[1]==tag]
    count_tag = len(tag_list)#total number of times the passed tag occurred in train_bag
    w_given_tag_list = [pair[0] for pair in tag_list if pair[0]==word[0]]

#now calculate the total number of times the passed word occurred as the passed tag.
    count_w_given_tag = len(w_given_tag_list)
 
     
    return (count_w_given_tag, count_tag)

In [ ]:
# compute  Transition Probability
def t2_given_t1(t2, t1, train_bag = train_set):
    tags = [pair[1] for pair in train_bag]
    count_t1 = len([t for t in tags if t==t1])
    count_t2_t1 = 0
    for index in range(len(tags)-1):
        if tags[index]==t1 and tags[index+1] == t2:
            count_t2_t1 += 1
    return (count_t2_t1, count_t1)

In [ ]:
# creating t x t transition matrix of tags, t= no of tags
# Matrix(i, j) represents P(jth tag after the ith tag)
 
tags_matrix = np.zeros((len(tags), len(tags)), dtype='float32')
for i, t1 in enumerate(list(tags)):
    for j, t2 in enumerate(list(tags)): 
        tags_matrix[i, j] = t2_given_t1(t2, t1)[0]/t2_given_t1(t2, t1)[1]
 
print(tags_matrix)

[[6.5690250e-04 6.7030865e-05 9.6020377e-01 3.8334955e-02 7.3733955e-04]
 [1.2524266e-04 1.2524266e-04 9.3869370e-01 5.8112592e-02 2.9432024e-03]
 [3.8031381e-02 4.0774336e-03 8.4304070e-01 1.0574059e-01 9.1096396e-03]
 [1.1575507e-03 6.0805196e-05 9.4543523e-01 5.1788010e-02 1.5584147e-03]
 [9.3112420e-04 1.3693003e-04 9.5785296e-01 3.8696427e-02 2.3825825e-03]]


In [ ]:
# convert the matrix to a df for better readability
#the table is same as the transition table shown in section 3 of article
tags_df = pd.DataFrame(tags_matrix, columns = list(tags), index=list(tags))
display(tags_df)

,U,T,N,S,H
U,0.000657,0.000067,0.960204,0.038335,0.000737
T,0.000125,0.000125,0.938694,0.058113,0.002943
N,0.038031,0.004077,0.843041,0.105741,0.009110
S,0.001158,0.000061,0.945435,0.051788,0.001558
H,0.000931,0.000137,0.957853,0.038696,0.002383


In [ ]:
def Viterbi(words, train_bag = train_set):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
     
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = [] 
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['N', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                 
            # compute emission and state probabilities
            emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
            state_probability = emission_p * transition_p    
            p.append(state_probability)
             
        pmax = max(p)
        # getting state for which probability is maximum
        state_max = T[p.index(pmax)] 
        state.append(state_max)
    return list(zip(words, state))

In [ ]:
# import random
# # Let's test our Viterbi algorithm on a few sample sentences of test dataset
# random.seed(1234)      #define a random seed to get same sentences when run multiple times
 
# # choose random 10 numbers
# rndom = [random.randint(1,len(test_set)) for x in range(10)]
 
# # list of 10 sents on which we test the model
# test_run = [test_set[i] for i in rndom]
 
# # list of tagged words
# test_run_base = [tup for sent in test_run for tup in sent]
 
# # list of untagged words
# test_set = [tup[0] for sent in test_run for tup in sent]a

In [ ]:
# test_set = train_set[4551343:]
# test_set
# print(test_set[:5])
# test_run_base = [tup for tup in test_set]

[('dá', 'N'), ('contae', 'S'), ('sa', 'N'), ('dáilcheantar', 'N'), (')', 'N')]


In [ ]:
# import time
# #Here We will only test 10 sentences to check the accuracy
# #as testing the whole training set takes huge amount of time
# start = time.time()
# tagged_seq = Viterbi(test_set)
# end = time.time()
# difference = end-start
 
# print("Time taken in seconds: ", difference)
 
# # accuracy
# check = [i for i, j in zip(tagged_seq, test_run_base) if i[1] == j[1]] 
 
# accuracy = len(check)/len(tagged_seq)
# print('Viterbi Algorithm Accuracy: ',accuracy*100)

Time taken in seconds:  39.84624171257019
Viterbi Algorithm Accuracy:  100.0
[(('dá', 'N'), 'N'), (('contae', 'S'), 'S'), (('sa', 'N'), 'N'), (('dáilcheantar', 'N'), 'N'), ((')', 'N'), 'N'), (('.', 'N'), 'N'), (('<S>', 'N'), 'N'), (('bí', 'S'), 'S'), (('ciall', 'N'), 'N'), (('lena', 'N'), 'N')]
[('dá', 'N'), ('contae', 'S'), ('sa', 'N'), ('dáilcheantar', 'N'), (')', 'N'), ('.', 'N'), ('<S>', 'N'), ('bí', 'S'), ('ciall', 'N'), ('lena', 'N')]


In [ ]:
# test = pd.read_csv('test.tsv',sep='\t')

In [ ]:
test=[]
test_file = open('test.tsv', 'r')
for line in test_file:
  value=line.rstrip("\n").split("\t")
  test.append(value)

In [ ]:
test

[['<S>', 'N'],
 ['<NUM>', 'N'],
 ['.', 'N'],
 ['—', 'N'],
 ['leasaítear', 'N'],
 ['leis', 'N'],
 ['seo', 'N'],
 ['cuid', 'N'],
 ['iv', 'N'],
 ['den', 'N'],
 ['príomh-acht', 'S'],
 ['—', 'N'],
 ['<S>', 'N'],
 ['cuige', 'S'],
 ['seo', 'N'],
 ['tá', 'N'],
 ['gá', 'N'],
 ['le', 'N'],
 ['comhtháthú', 'N'],
 ['a', 'N'],
 ['déanamh', 'S'],
 ['ar', 'N'],
 ['na', 'N'],
 ['struchtúir', 'N'],
 ['comhairleacha', 'S'],
 ['agus', 'N'],
 ['tacaíochta', 'S'],
 ['atá', 'N'],
 ['ar', 'N'],
 ['fáil', 'N'],
 ['do', 'N'],
 ['scoileanna', 'N'],
 ['gaeltachta', 'N'],
 ['sa', 'N'],
 ['caoi', 'S'],
 ['is', 'N'],
 ['gur', 'N'],
 ['féidir', 'N'],
 ['an', 'N'],
 ['beagán', 'N'],
 ['saineolais', 'N'],
 ['atá', 'N'],
 ['ar', 'N'],
 ['fáil', 'N'],
 ['a', 'N'],
 ['tabhairt', 'S'],
 ['le', 'N'],
 ['céile', 'S'],
 [',', 'N'],
 ['an', 'N'],
 ['úsáid', 'N'],
 ['is', 'N'],
 ['fearr', 'N'],
 ['a', 'N'],
 ['baint', 'S'],
 ['as', 'N'],
 [',', 'N'],
 ['agus', 'N'],
 ['é', 'N'],
 ['a', 'N'],
 ['cur', 'S'],
 ['ar', 'N'],
 ['fái

In [ ]:
test1 = pd.DataFrame()
test1['test']= test

In [ ]:
test1

,test
0,"[<S>, N]"
1,"[<NUM>, N]"
2,"[., N]"
3,"[—, N]"
4,"[leasaítear, N]"
...,...
999995,"[d'fhás, N]"
999996,"[martin, N]"
999997,"[walser, N]"
999998,"[aníos, N]"


In [ ]:
words = []
pos = []
for col in range(len(test1['test'])):
  words.append(test1['test'][col][0])
  pos.append(test1['test'][col][1])

In [ ]:
test1['words'] = words
test1['pos'] = pos

In [ ]:
test1 = test1.drop('test',axis=1)

In [ ]:
test1

,words,pos
0,<S>,N
1,<NUM>,N
2,.,N
3,—,N
4,leasaítear,N
...,...,...
999995,d'fhás,N
999996,martin,N
999997,walser,N
999998,aníos,N


In [ ]:
# UNIGRAM TUPLE LIST

m = test1['words'].tolist()
n = test1['pos'].tolist()
liss=[]
for i in range(len(m)):
   liss.append((m[i],n[i]))
  


In [ ]:
test_set = liss[:100]

In [ ]:
print(test_set[:5])
test_run_base = [tup for tup in test_set]

[('<S>', 'N'), ('<NUM>', 'N'), ('.', 'N'), ('—', 'N'), ('leasaítear', 'N')]


In [ ]:
import time
#Here We will only test 10 sentences to check the accuracy
#as testing the whole training set takes huge amount of time
start = time.time()
tagged_seq = Viterbi(test_set)
end = time.time()
difference = end-start
 
print("Time taken in seconds: ", difference)
 
# accuracy
check = [i for i, j in zip(tagged_seq, test_run_base) if i[1] == j[1]] 
 
accuracy = len(check)/len(tagged_seq)
print('Viterbi Algorithm Accuracy: ',accuracy*100)

Time taken in seconds:  504.9579608440399
Viterbi Algorithm Accuracy:  96.0


In [ ]:
#ignore this part
import time
import threading
def printit():
  threading.Timer(5, printit).start()
  start = time.time()
  tagged_seq = Viterbi(test_set)
  end = time.time()
  difference = end-start
  print("Time taken in seconds: ", difference)
  # accuracy
def accuracy():
    check = [i for i, j in zip(tagged_seq, test_run_base) if i[1] == j[1]] 
    accuracy = len(check)/len(tagged_seq)
    print('Viterbi Algorithm Accuracy: ',accuracy*100)
    return accuracy

printit()

KeyboardInterrupt: ignored